<a href="https://colab.research.google.com/github/comeduguy/Python-Example/blob/master/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C_%EC%9E%90%EC%B9%98%EA%B5%AC%EB%B3%84_%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80_%ED%98%84%ED%99%A9_%EC%8B%9C%EA%B0%81%ED%99%94%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 준비하기
[서울시 권역별 실시간 대기환경 현황](http://data.seoul.go.kr/dataList/OA-2219/S/1/datasetView.do)

##가. json 파일 다운로드 및 Colab 업로드
- 예시파일은 2021년 3월 30일 12:00 데이터
- 파일명을 ori_data.json으로 변경 후 업로드

In [ ]:
# 파일 업로드하기(json)
from google.colab import files
uploaded = files.upload()

##나. JSON 파일 분석


In [ ]:
# json 파일을 다루기 위해 json 모듈 import
import json

In [ ]:
# 파일 읽기
file_json = open('ori_data.json', 'r')

data_json = json.load(file_json)

In [ ]:
print(data_json)

In [ ]:
# 파일 내용 분석
print(data_json['DESCRIPTION'])
print(data_json['DATA'])

In [ ]:
for i in range(len(data_json['DATA'])):
    print(data_json['DATA'][i]['msrste_nm'])

##다. 미세먼지 딕셔너리 만들기

In [ ]:
# 자치구별 미세먼지 농도 딕셔너리 생성
data = {}

N = len(data_json['DATA'])

for i in range(N):
    data[data_json['DATA'][i]['msrste_nm']] = data_json['DATA'][i]['pm10']

print(data)

In [ ]:
# 자치구를 입력받아 미세먼지 농도 출력
NM = input('미세먼지를 출력하려는 구 이름을 입력하세요.: ')

print(data[NM])

In [ ]:
# 구명 변경위한 csv 활용 위해 csv 모듈 import
import csv

- 구명 영문-한글 저장 파일 [gu.csv](https://drive.google.com/file/d/1U9dm46JQ6YlLdLslW63gttMCqIOMgeEt/view?usp=sharing)
- 파일 다운로드 후 Colab 업로드

In [ ]:
# 자치구명 영어 매칭을 위해 파일 읽어오기
file_csv = open('gu.csv', 'r', encoding='utf-8')

gu_data = csv.reader(file_csv)

In [ ]:
# 구별 영문명과 미세먼지 농도를 새로운 딕셔너리로 생성
data_en = {}

for row in gu_data:
    if row[0] == '\ufeffJongno-gu':
        data_en['Jongno-gu'] = data[row[1]]
    elif row[0] == 'Yeongdeungpo-gu':
        data_en['Yeongdeungpo-gu_1_'] = data[row[1]]
    else:
        data_en[row[0]] = data[row[1]]

print(data_en)

#2. 데이터 시각화 1 - SVG

##가. SVG 파일을 활용한 데이터 시각화

In [ ]:
# svg 파일을 변경하기 위해 Beautifulsoup 모듈 import
from bs4 import BeautifulSoup

In [ ]:
# BeautifulSoup을 이용하여 SVG 파일을 HTML 형태로 변환
file_svg = open('Seoul_districts.svg', 'r')

data_soup = BeautifulSoup(file_svg.read(), 'html.parser')

print(data_soup)

In [ ]:
# 자치구 채우기 색상 변경 테스트
for row in data_soup.find_all('path'):
    row['fill'] = "#abcdef"

print(data_soup)

In [ ]:
# 색상 변경한 결과를 HTML 파일로 저장
file_html = open('result.html', 'w')
file_html.write(data_soup.prettify())
file_html.close()

- 미세먼지 농도별로 색상을 구분하여 적용
- [ColorBrewer 2.0](https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)

In [ ]:
# ['#fef0d9','#fdcc8a','#fc8d59','#d7301f'] 좋음 -> 매우 나쁨
color = ['#fef0d9','#fdcc8a','#fc8d59','#d7301f']
for row in data_soup.find_all('path'):
    if int(data_en[row['id']]) >= 150:
        gr = 3
    elif int(data_en[row['id']]) >= 80:
        gr = 2
    elif int(data_en[row['id']]) >= 30:
        gr = 1
    else:
        gr = 0
    row['fill'] = color[gr]

file_html = open('result.html', 'w')
file_html.write(data_soup.prettify())
file_html.close()

##나. OpenAPI를 활용한 실시간 정보 조회

In [ ]:
# 실시간으로 정보를 조회하기 위해 requests 모듈 import
import requests

In [ ]:
# 실시간 정보 조회 결과를 BeautifulSoup으로 HTML 형식으로 변환
url = 'http://openAPI.seoul.go.kr:8088/6270756b78636f6d3535466a6a4e50/xml/RealtimeCityAir/1/25/'

data_response = requests.get(url)

data_soup = BeautifulSoup(data_response.content, 'html.parser')

print(data_soup)

In [ ]:
# 자치구명과 미세먼지 농도 값으로 data 딕셔너리 생성
PM10 = data_soup.find_all('pm10')
MSRSTE_NM = data_soup.find_all('msrste_nm')

data = {}

for i in range(len(PM10)):
    data[MSRSTE_NM[i].text] = int(PM10[i].text)
    print(MSRSTE_NM[i].text, data[MSRSTE_NM[i].text])

In [ ]:
# 구별 영문명과 미세먼지 농도를 새로운 data_en 딕셔너리로 생성
data_en = {}

for row in gu_data:
    if row[0] == '\ufeffJongno-gu':
        data_en['Jongno-gu'] = data[row[1]]
    elif row[0] == 'Yeongdeungpo-gu':
        data_en['Yeongdeungpo-gu_1_'] = data[row[1]]
    else:
        data_en[row[0]] = data[row[1]]

print(data_en)

In [ ]:
# ['#fef0d9','#fdcc8a','#fc8d59','#d7301f'] 좋음 -> 매우 나쁨
color = ['#fef0d9','#fdcc8a','#fc8d59','#d7301f']

# BeautifulSoup을 이용하여 SVG 파일을 HTML 형태로 변환
file_svg = open('Seoul_districts.svg', 'r')

data_soup = BeautifulSoup(file_svg.read(), 'html.parser')

for row in data_soup.find_all('path'):
    if int(data_en[row['id']]) >= 150:
        gr = 3
    elif int(data_en[row['id']]) >= 80:
        gr = 2
    elif int(data_en[row['id']]) >= 30:
        gr = 1
    else:
        gr = 0
    row['fill'] = color[gr]

file_html = open('result.html', 'w')
file_html.write(data_soup.prettify())
file_html.close()

# 3. 데이터 시각화2 - Folium

In [1]:
import folium

In [40]:
ori_loc = [37.5744458740, 126.9094147806]
TILES = 'OpenStreetMap'
# “Mapbox Bright” (Limited levels of zoom for free tiles)
# “Mapbox Control Room” (Limited levels of zoom for free tiles)
# “Stamen” (Terrain, Toner, and Watercolor)
# “Cloudmade” (Must pass API key)
# “Mapbox” (Must pass API key)
# “CartoDB” (positron and dark_matter)

m = folium.Map(ori_loc, tiles=TILES, width=640, height=480, zoom_start= 15)

m

In [ ]:
# 